<img align="left" src="imgs/fonduer-logo.png" width="100px" style="margin-right:20px">

# Tutorial: Navigating the Fonduer Data Model

## Running locally?

If you're running this tutorial interactively on your own machine, you'll need to create a new PostgreSQL database named `intro_data_model`.

If you have already the database `intro_data_model` in your postgresql, please uncomment the first line to drop it. Otherwise, download our database snapshots by executing `./download_data.sh` in the intro tutorial directory.

In [ ]:
#! dropdb --if-exists intro_data_model
! createdb intro_data_model
! psql intro_data_model < data/intro_data_model.sql > /dev/null

# The Fonduer Data Model API
_Complete Fonduer API documentation is available on [Read the Docs](https://fonduer.readthedocs.io)_

## The Fonduer Data Model
The Fonduer Data Model serves two high-level purposes. First, it allows Fonduer to capture the immense data variety of richly formatted data in a unified representation. Second, it allows users to provide multimodal supervision that leverages document-level context. Nearly everything in the Fonduer pipeline uses information stored in the Fonduer Data Model. The Fonduer Data Model is heirarchical, as shown below.

<img src="imgs/dag.png" width="250px">

Each box represents a `Context` object, which are units that a document can be broken down into. For example, a `Sentence` can be part of a `Paragraph` in a `Cell` in a `Table`. The default `Context` objects provided by Fonduer are shown above.

To explore the data model, we first load the Fonduer `Meta` class, which creates a connection with our PostgreSQL database.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import sys
import logging
from pprint import pprint

from fonduer.parser.models import Document, Sentence, Table, Figure
from fonduer import Meta, init_logging


ATTRIBUTE = "intro_data_model"
conn_string = f'postgresql://localhost:5432/{ATTRIBUTE}'

# Configure logging for Fonduer
init_logging(log_dir="logs")

session = Meta.init(conn_string).Session()

# Print metadata about the tutorial corpus
print(f"Num Docs: {session.query(Document).count()}")
print(f"Num Sentences: {session.query(Sentence).count()}")
print(f"Num Tables: {session.query(Table).count()}")
print(f"Num Figures: {session.query(Figure).count()}")

### Parsing Documents into the Data Model

The process of parsing input documents containing richly formatted data into Fonduer's data model typically consists of two steps. First, documents can be run through a [Preprocessor](https://fonduer.readthedocs.io/en/stable/user/parser.html#preprocessors), and then fed through the Fonduer Parser to turn the input documents into the directed acyclic graph shown above. For more information about parsing documents, please check out our [end-to-end tutorials](../hardware/max_storage_temp_tutorial.ipynb). The rest of this tutorial assumes that your documents have already been parsed, and instead focuses on the data model itself.


### Navigating the Data Model

With the `session` object, you can make queries for any of `Context` objects in the data model using the [SQLAlchemy Query API](http://docs.sqlalchemy.org/en/latest/orm/query.html). For example, if we wanted to get a list of all of the Documents in the database, we can issue the following query, which gets all of the Documents ordered by name.

In [ ]:
docs = session.query(Document).order_by(Document.name).all()

pprint(docs)

Similarly, if we wanted to inspect all of the Sentences parsed, we could issue the following query.

In [ ]:
sentences = session.query(Sentence).order_by(Sentence.position).all()
pprint(sentences[:5])

In addition to simply querying for specific `Context` objects in the database, we can also navigate the data model heirarchy from each `Context` object. The core `Context` objects are described on [Read the Docs](https://fonduer.readthedocs.io/en/stable/user/parser.html#multimodal-data-model). Let's see some examples.

#### Ex. 1: Getting a Sentence's Document

In [ ]:
bc182 = sentences[0].document
print(bc182)

#### Ex. 2: Iterate over all a Document's Sentences

In [ ]:
pprint([sentence for sentence in bc182.sentences][:10])

#### Ex. 3: Find the first Sentence in the first Table in the Document

In [ ]:
print(bc182.tables[0].sentences[0])

#### Ex. 4: Inspect the html attributes of a Sentence

In [ ]:
pprint(bc182.tables[0].sentences[0].html_attrs)

#### Ex. 5: Listing the attributes of a Context Object

If you forget the API and want to inspect the attributes of a particular Context object, you can just call dir() for a full list if you're working interactively. Otherwise, we'd recommend referring to [Read the Docs](https://fonduer.readthedocs.io/en/stable/user/parser.html#multimodal-data-model).


In [ ]:
dir(bc182.tables[0].sentences[0])

#### Ex. 6: Inspect the Cell containing a particular Sentence

In [ ]:
print(bc182.tables[0].sentences[0].cell)

#### Ex. 7: Iterate over all the Cells in a particular row

Note that Cells in a Table can span rows or columns. Thus, each Cell's row and column is indicated with `(row_start, row_end)` and `(col_start, col_end)`.

In [ ]:
table = bc182.tables[0].sentences[0].cell.table
pprint([cell for cell in table.cells if cell.row_start == 1])

## Summary
The Fonduer Data Model is a heirarchical representation of the input documents. Using the data model APIs, you can traverse anywhere in the data model and inspect the attributes of your data. Visit [Fonduer's Documentation](https://fonduer.readthedocs.io/en/stable/user/parser.html#multimodal-data-model) for a reference of the API.